# Create a Pipeline

In [ ]:
import os
import boto3
from botocore.client import Config

import kfp
from kfp import dsl
from kfp.dsl.types import Float, Integer, String

## Upload the 'getting-started' notebook to a bucket

In [ ]:
s3 = boto3.client(
    "s3",
    endpoint_url="http://minio-service.kubeflow:9000",
    aws_access_key_id="minio",
    aws_secret_access_key="minio123",
    config=Config(signature_version="s3v4"),
    region_name="us-east-1"
)

filename = "01-getting-started.ipynb"
with open(filename, "rb") as fp:
    s3.upload_fileobj(fp, "mlpipeline", filename)

## Create a pipeline that runs our notebook

In [ ]:
@dsl.pipeline(
    name="train_pipeline",
    description="""A pipeline that runs jupyter notebooks using Papermill.""",
)
def train(
    epochs: Integer = 10,
    dropout: Float = 0.0,
    weight_decay: Float  = 0.0,
    n_chunks: Float = 700,
    learning_rate: Float = 0.03,
    optimizer: String = "SGD"
):
    notebook_path = "s3://mlpipeline/01-getting-started.ipynb"
    output_path = "-"
    dsl.ContainerOp(
        name="Train",
        image="gcr.io/kubeflow-aifest-2019/jupyter",
        container_kwargs={
            "image_pull_policy": "IfNotPresent",
            "resources": {
                "limits": {
                    "cpu": "1.0",
                    "memory": "5Gi"
                },
                "requests": {
                    "cpu": "0.1",
                    "memory": "1Gi"
                }
            }
        },
        command=[
            "papermill", notebook_path, output_path,
            "--report-mode",
            "-p", "epochs", epochs,
            "-p", "dropout", dropout,
            "-p", "weight_decay", weight_decay,
            "-p", "n_chunks", n_chunks,
            "-p", "learning_rate", learning_rate,
            "-p", "optimizer", optimizer,
        ],
        file_outputs={
            "mlpipeline-ui-metadata": "/mlpipeline-ui-metadata.json",
            "mlpipeline-metrics": "/mlpipeline-metrics.json"
        }
    )

In [ ]:
kfp.compiler.Compiler().compile(train, "training.zip")

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<a href="/notebook/anonymous/my-notebook-server/files/kubeflow-aiops-workshop/notebooks/training.zip?download=1">Download</a>'))